In [20]:
import pandas as pd
import numpy as np

from pycaret.classification import *

ModuleNotFoundError: No module named 'pycaret'

<a href="https://pub.towardsai.net/what-are-baseline-models-and-benchmarking-for-machine-learning-why-we-need-them-affe0714cd07"> Link </a>

This notebook focuses on the methodology to use baseline models as a starting point & reference in machine learning for further model development.<br>
In general, this reference point is important in the data science process to explain how the developed models perform (good or bad) and the baseline model's score serves as threshold for this evaluation.<br>

There are 3 types of baseline models:
1) Random Baseline Models 
2) ML Baseline Models 
3) Automated ML Baseline Models

---

## 1. Random Baseline Models

If data is not predictable, the best baseline model is a dummy classifier or dummy regressor.<br>
This way, in comparison to the baseline model, it's possible to evaluate the performance of the developed model.

In [3]:
np.random.seed(0)

random_dim = (1000, 3)
random_X = np.random.random(random_dim)
random_reg_y = np.random.randint(random_dim[1], size = random_dim[0])

In [16]:
train_clf = np.concatenate([random_X, random_reg_y.reshape(random_dim[0], 1)], axis = 1)
col_list = [str(i+1) for i in range(random_dim[1])]
col_list.append("target")

train_clf = pd.DataFrame(train_clf, columns = col_list)

train_clf["target"] = train_clf["target"].astype("str")

In [19]:
train_clf.sample(5)

,1,2,3,target
831,0.377430,0.670239,0.471664,2.0
394,0.664666,0.423054,0.198991,0.0
954,0.309050,0.069587,0.365474,0.0
696,0.705307,0.365676,0.395411,1.0
489,0.196451,0.671528,0.842973,2.0
